Run the next cell if your APIM is using a certificate issues from a private certificate authority. This variable will tell the httpx library the OpenAI SDK uses
to trust the certificate bundle. If you don't do this, the OpenAI SDK will not work.

In [ ]:
set SSL_CERT_FILE=C:\Users\localadmin\Desktop\code\root.pem

Import the necessary libraries

In [ ]:
import logging
import sys
import os
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
from openai import AzureOpenAI, OpenAI

Configure required environmental variables. You should include the service principal identity credentials you created as a pre-requisite to this lab.

In [ ]:
AZURE_CLIENT_ID="XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"
AZURE_CLIENT_SECRET="somesecret"
AZURE_TENANT_ID="XXXXXXXX-XXXX-XXXX-XXXX-XXXXXXXXXXXX"
AZURE_OPENAI_ENDPOINT = "https://apim-example.apim.myapimdomain.com"
API_VERSION = "2025-04-01-preview"
DEPLOYMENT_NAME = "gpt-4o"



Setup logging

logging.basicConfig(
    level=logging.ERROR,
    format='%asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[logging.StreamHandler(sys.stdout)]
)

Issue chat completion. This includes the usage of the UserSecurityContext which is a [required attribute for Defender for AI and Purview to surface identity data associated to malicious prompts](https://journeyofthegeek.com/2025/10/28/defender-for-ai-and-user-context/).

In [ ]:
client = AzureOpenAI(
        api_version = API_VERSION,
        azure_endpoint= AZURE_OPENAI_ENDPOINT,
        azure_ad_token_provider=token_provider
)

user_security_context = {
        "end_user_id": "2e69d9f2-b5b3-482b-9c15-XXXXXXXXXXXX",
        "source_ip": "10.52.7.4",
        "application_name": "My RAG Application"
}
response = client.chat.completions.create(
model=deployment_name,
messages=[
        {"role":"system",
          "content": "You are a helpful assistant."},
        {"role": "user",
         "content": "What color is the sky?"}
],
max_tokens=4096,
extra_body={"user_security_context": user_security_context }
)

print(response.choices[0].message.content)